<a href="https://colab.research.google.com/github/flydragon2018/colab_notebooks/blob/main/yolov5_high_resolution_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Yolov5 high resolution training

### Major modification
* img=3600
* mixup=0.5
* fliplr: 0.5

### Hardware to reproduce
* RTX3090

In [1]:
!nvidia-smi

Mon Jan 17 07:57:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/mydrive', force_remount=True)

Mounted at /content/mydrive


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from shutil import copyfile

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [4]:
#train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')

train = pd.read_csv('/content/mydrive/MyDrive/kaggle/reef/train.csv')


train['pos'] = train.annotations != '[]'

In [5]:
!mkdir -p ./yolo_data/fold1/images/val
!mkdir -p ./yolo_data/fold1/images/train

!mkdir -p ./yolo_data/fold1/labels/val
!mkdir -p ./yolo_data/fold1/labels/train

In [6]:
'''
fold = 1

annos = []
for i, x in train.iterrows():
    if x.video_id == fold:
        mode = 'val'
    else:
        # train
        mode = 'train'
        if not x.pos: continue
        # val
    copyfile(f'/content/mydrive/MyDrive/kaggle/reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
                f'./yolo_data/fold{fold}/images/{mode}/{x.image_id}.jpg')
    if not x.pos:
        continue
    r = ''
    anno = eval(x.annotations)
    for an in anno:
#            annos.append(an)
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
    with open(f'./yolo_data/fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)
 '''

"\nfold = 1\n\nannos = []\nfor i, x in train.iterrows():\n    if x.video_id == fold:\n        mode = 'val'\n    else:\n        # train\n        mode = 'train'\n        if not x.pos: continue\n        # val\n    copyfile(f'/content/mydrive/MyDrive/kaggle/reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',\n                f'./yolo_data/fold{fold}/images/{mode}/{x.image_id}.jpg')\n    if not x.pos:\n        continue\n    r = ''\n    anno = eval(x.annotations)\n    for an in anno:\n#            annos.append(an)\n        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,\n                                        (an['y'] + an['height'] / 2) / 720,\n                                        an['width'] / 1280, an['height'] / 720)\n    with open(f'./yolo_data/fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:\n        fp.write(r)\n "

In [7]:
#!cp -rf  /content/yolo_data  /content/mydrive/MyDrive/

In [8]:
!cp -rf /content/mydrive/MyDrive/yolo_data  /content/

In [9]:
hyps = '''
# YOLOv5 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.5  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
'''

In [10]:
data = '''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../yolo_data/fold1/  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['reef']  # class names


# Download script/URL (optional)
# download: https://ultralytics.com/assets/coco128.zip
'''

In [11]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 10558, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 10558 (delta 0), reused 3 (delta 0), pack-reused 10555
Receiving objects: 100% (10558/10558), 10.75 MiB | 11.01 MiB/s, done.
Resolving deltas: 100% (7292/7292), done.


In [12]:
with open('./yolov5/data/reef_f1_naive.yaml', 'w') as fp:
    fp.write(data)
with open('./yolov5/data/hyps/hyp.heavy.2.yaml', 'w') as fp:
    fp.write(hyps)

In [13]:
%cd yolov5

/content/yolov5


In [14]:
!ls data/

Argoverse.yaml	      hyps		  scripts	 xView.yaml
coco128.yaml	      images		  SKU-110K.yaml
coco.yaml	      Objects365.yaml	  VisDrone.yaml
GlobalWheat2020.yaml  reef_f1_naive.yaml  VOC.yaml


In [15]:
!pip install -U PyYAML

     |████████████████████████████████| 596 kB 4.0 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [16]:
#!pip install   albumentations==1.0.3

In [ ]:
!python train.py --img 3008  --batch 2 --epochs 16 --data reef_f1_naive.yaml --weights yolov5s6.pt --name l6_3600_uflip_vm5_f1 --hyp data/hyps/hyp.heavy.2.yaml

train: weights=yolov5s6.pt, cfg=, data=reef_f1_naive.yaml, hyp=data/hyps/hyp.heavy.2.yaml, epochs=16, batch_size=2, imgsz=3008, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=l6_3600_uflip_vm5_f1, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-193-gdb1f83b torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, de

In [ ]:
!ls

In [ ]:
!mkdir -p  /content/mydrive/MyDrive/yolov5_train/0118

In [ ]:
!cp -rf /content/yolov5/runs/train  /content/mydrive/MyDrive/yolov5_train/0118